In [6]:
import requests
import json
import base64
import pandas as pd
import re

# StubHub

## Authentication

In [15]:
with open('./NYCDSA-Capstone-Project/passwords.txt') as passwords:
    text = passwords.readlines()

app_token = re.search("'.*", text[0]).group().replace("'", "")

consumer_key = re.search("'.*", text[2]).group().replace("'", "")
consumer_secret = re.search("'.*", text[3]).group().replace("'", "")

stubhub_username = re.search("'.*", text[5]).group().replace("'", "")
stubhub_password = re.search("'.*", text[6]).group().replace("'", "")

In [17]:
combo = consumer_key + ':' + consumer_secret
basic_authorization_token = base64.b64encode(combo.encode('utf-8'))

In [18]:
headers = {
        'Content-Type':'application/x-www-form-urlencoded',
        'Authorization':'Basic '+basic_authorization_token.decode('utf-8'),}

body = {
        'grant_type':'password',
        'username':stubhub_username,
        'password':stubhub_password,
        'scope': 'PRODUCTION'}

url = 'https://api.stubhub.com/login'
r = requests.post(url, headers=headers, data=body)

token_respoonse = r.json()
access_token = token_respoonse['access_token']
user_GUID = r.headers['X-StubHub-User-GUID']

200

In [19]:
headers['Authorization'] = 'Bearer ' + access_token
headers['Accept'] = 'application/json'
headers['Accept-Encoding'] = 'application/json'

## Get List of Events

In [124]:
city_list = ['San Francisco', 'Oakland', 'Berkeley', 'San Jose',
             'New York', 'Brooklyn', 'Bronx', 'Flushing', 'East Rutherford',
            'Washington, DC', 'Vienna',
             'Chicago', 'Rosemont', 'Evanston',
            'Los Angeles', 'Hollywood', 'West Hollywood', 'Pasadena',
            'Boston', 'Medford']

city_filt = '"' + '"  |"'.join(city_list) + '"'

In [125]:
def parse_event_list(events):
    for event in events:
        id_.append(event['id'])
        name.append(event['name'])
        venue.append(event['venue']['name'])
        date.append(event['eventDateLocal'])
        n_listings.append(event['ticketInfo']['totalListings'])
        city.append(event['venue']['city'])
        state.append(event['venue']['state'])
        
events_url = 'https://api.stubhub.com/search/catalog/events/v3'

filt = {'city': city_filt, 'minAvailableTickets': 1, 'q': 'concert', 'sort': 'eventDateLocal',
        'start': 0, 'rows': 500} #, 'fieldList': 'id,ticketInfo'}
        
events_r = requests.get(events_url, headers = headers, params = filt).json()
events = events_r['events']
n_found = events_r['numFound']

id_, name, venue, date, n_listings, city, state = [], [], [], [], [], [], []
parse_event_list(events)

while filt['start'] < n_found:
    events = requests.get(events_url, headers = headers, params = filt).json()['events']
    parse_event_list(events)
    filt['start'] += 500

In [126]:
listings_df = pd.DataFrame({'id': id_, 'name': name, 'venue': venue, 'date': date, 'n_listings': n_listings,
                           'city': city, 'state': state})
listings_df = listings_df.drop_duplicates(subset='id')

parking_passes = listings_df.name.apply(lambda n: re.search('parking passes only', n.lower()) != None)
listings_df = listings_df[~parking_passes]

listings_df.head()

,id,name,venue,date,n_listings,city,state
0,103626023,Phil Lesh and The Terrapin Family Band,Central Park Summerstage,2018-09-05T18:00:00-0400,5,New York,NY
1,103623857,Parkway Drive with August Burns Red,Riviera Theatre Chicago,2018-09-05T18:00:00-0500,2,Chicago,IL
2,103733179,SCANDAL - Japanese Band,PlayStation Theater,2018-09-05T19:00:00-0400,5,New York,NY
3,103726234,Lil Baby,Irving Plaza,2018-09-05T19:00:00-0400,7,New York,NY
4,103358411,Juanes,Hollywood Bowl,2018-09-05T19:00:00-0700,44,Hollywood,CA


In [127]:
import math

print('calls: ', listings_df.n_listings.apply(lambda x: math.ceil(x/250)).sum())
print('events: ', len(listings_df))
print('records: ', listings_df.n_listings.sum())
print('hours: ', listings_df.n_listings.apply(lambda x: math.ceil(x/250)).sum()/(10*60))

calls:  3891
events:  3691
records:  166948
hours:  6.485


## Inventory

In [42]:
inventory_url = 'https://api.stubhub.com/search/inventory/v2'

headers['Authorization'] = 'Bearer ' + access_token
headers['Accept'] = 'application/json'
headers['Accept-Encoding'] = 'application/json'

## Enter event ID
eventid = '103611294'
data = {'eventid': eventid, 'start': 0, 'rows': 250}

## GET request and change to Pandas dataframe
inventory = requests.get(inventory_url, headers=headers, params=data)
inv = inventory.json()

listing_df = pd.DataFrame(inv['listing'])

In [44]:
inv

{'eventId': 103611294,
 'totalListings': 256,
 'totalTickets': 1083,
 'minQuantity': 1,
 'maxQuantity': 24,
 'listing': [{'listingId': 1327971664,
   'currentPrice': {'amount': 103.3, 'currency': 'USD'},
   'listingPrice': {'amount': 84.0, 'currency': 'USD'},
   'sectionId': 1693365,
   'row': 'O',
   'quantity': 6,
   'sellerSectionName': '425',
   'sectionName': 'Upper Concourse 425',
   'zoneId': 244507,
   'zoneName': 'Upper Concourse',
   'listingAttributeList': [201],
   'listingAttributeCategoryList': [1],
   'deliveryTypeList': [2],
   'deliveryMethodList': [2],
   'isGA': 0,
   'dirtyTicketInd': False,
   'splitOption': '1',
   'ticketSplit': '2',
   'splitVector': [2, 4, 6],
   'sellerOwnInd': 0,
   'score': 0.0},
  {'listingId': 1359008510,
   'currentPrice': {'amount': 104.5, 'currency': 'USD'},
   'listingPrice': {'amount': 85.0, 'currency': 'USD'},
   'sectionId': 1693477,
   'row': 'O',
   'quantity': 2,
   'sellerSectionName': '408',
   'sectionName': 'Upper Concourse 4

In [43]:
listing_df

,businessGuid,currentPrice,deliveryMethodList,deliveryTypeList,dirtyTicketInd,faceValue,isGA,listingAttributeCategoryList,listingAttributeList,listingId,...,seatNumbers,sectionId,sectionName,sellerOwnInd,sellerSectionName,splitOption,splitVector,ticketSplit,zoneId,zoneName
0,NaN,"{'amount': 103.3, 'currency': 'USD'}",[2],[2],False,NaN,0,[1],[201],1327971664,...,NaN,1693365,Upper Concourse 425,0,425,1,"[2, 4, 6]",2,244507,Upper Concourse
1,NaN,"{'amount': 104.5, 'currency': 'USD'}",[2],[2],False,NaN,0,[1],[201],1359008510,...,NaN,1693477,Upper Concourse 408,0,408,1,[2],2,244507,Upper Concourse
2,321EDF421AF5217AE0540010E056A2AC,"{'amount': 104.5, 'currency': 'USD'}",[2],[2],False,"{'amount': 78.7, 'currency': 'USD'}",0,[1],[201],1360170775,...,NaN,1693430,Upper Concourse 424,0,424,2,"[1, 2, 3, 4, 6]",1,244507,Upper Concourse
3,NaN,"{'amount': 104.5, 'currency': 'USD'}","[22, 23, 24]",[5],False,NaN,0,NaN,NaN,1359781233,...,General Admission,1693522,Upper Concourse 418,0,Upper Concourse 418,2,[2],1,244507,Upper Concourse
4,NaN,"{'amount': 104.5, 'currency': 'USD'}",[2],[2],False,NaN,0,NaN,NaN,1358008149,...,"16,17",1693400,Upper Concourse 403,0,Upper Concourse 403,1,[2],2,244507,Upper Concourse
5,NaN,"{'amount': 105.7, 'currency': 'USD'}",[2],[2],False,NaN,0,[1],[201],1328094375,...,NaN,1693477,Upper Concourse 408,0,408,1,"[2, 4, 6]",2,244507,Upper Concourse
6,NaN,"{'amount': 106.32, 'currency': 'USD'}",[2],[2],False,NaN,0,NaN,NaN,1359512620,...,NaN,1693497,Upper Concourse 423,0,423,2,[2],1,244507,Upper Concourse
7,NaN,"{'amount': 106.9, 'currency': 'USD'}",[2],[2],False,NaN,0,[1],[201],1327994775,...,NaN,1693365,Upper Concourse 425,0,425,1,"[2, 4, 6]",2,244507,Upper Concourse
8,321EDF421AF5217AE0540010E056A2AC,"{'amount': 106.9, 'currency': 'USD'}",[2],[2],False,"{'amount': 79.7, 'currency': 'USD'}",0,[1],[201],1358998940,...,NaN,1693355,Upper Concourse 427,0,427,2,[2],1,244507,Upper Concourse
9,NaN,"{'amount': 107.07, 'currency': 'USD'}",[2],[2],False,NaN,0,[1],[201],1357318533,...,NaN,1693494,Upper Concourse 428,0,428,1,"[2, 4]",2,244507,Upper Concourse


In [13]:
data['start'] += 100

while data['start'] <= inv['totalListings']: 
    
    inventory = requests.get(inventory_url, headers=headers, params=data)
    inv = inventory.json()
    
    listing_df = pd.concat([listing_df, pd.DataFrame(inv['listing'])], axis=0, sort = False)
    
    data['start'] += 100

/Users/johnmcglynn/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [14]:
listing_df.head()

,businessGuid,currentPrice,deliveryMethodList,deliveryTypeList,dirtyTicketInd,faceValue,isGA,listingAttributeCategoryList,listingAttributeList,listingId,...,seatNumbers,sectionId,sectionName,sellerOwnInd,sellerSectionName,splitOption,splitVector,ticketSplit,zoneId,zoneName
0,NaN,"{'amount': 103.3, 'currency': 'USD'}",[2],[2],False,NaN,0,[1],[201],1327971664,...,NaN,1693365,Upper Concourse 425,0,425,1,"[2, 4, 6]",2,244507,Upper Concourse
1,NaN,"{'amount': 103.3, 'currency': 'USD'}",[2],[2],False,NaN,0,NaN,NaN,1360024576,...,"21,20",1693477,Upper Concourse 408,0,Upper Concourse 408,0,[2],2,244507,Upper Concourse
2,NaN,"{'amount': 104.5, 'currency': 'USD'}",[2],[2],False,NaN,0,[1],[201],1359008510,...,NaN,1693477,Upper Concourse 408,0,408,1,[2],2,244507,Upper Concourse
3,NaN,"{'amount': 104.5, 'currency': 'USD'}","[22, 23, 24]",[5],False,NaN,0,NaN,NaN,1359781233,...,General Admission,1693522,Upper Concourse 418,0,Upper Concourse 418,2,"[1, 2, 4]",1,244507,Upper Concourse
4,NaN,"{'amount': 105.7, 'currency': 'USD'}",[2],[2],False,NaN,0,[1],[201],1328094375,...,NaN,1693477,Upper Concourse 408,0,408,1,"[2, 4, 6]",2,244507,Upper Concourse


In [15]:
listing_df.columns

Index(['businessGuid', 'currentPrice', 'deliveryMethodList',
       'deliveryTypeList', 'dirtyTicketInd', 'faceValue', 'isGA',
       'listingAttributeCategoryList', 'listingAttributeList', 'listingId',
       'listingPrice', 'quantity', 'row', 'score', 'seatNumbers', 'sectionId',
       'sectionName', 'sellerOwnInd', 'sellerSectionName', 'splitOption',
       'splitVector', 'ticketSplit', 'zoneId', 'zoneName'],
      dtype='object')

In [163]:
listing_df[['row', 'seatNumbers', 'sectionId', 'sectionName']]

,row,seatNumbers,sectionId,sectionName
0,N,NaN,1693477,Upper Concourse 408
1,M,NaN,1693494,Upper Concourse 428
2,O,NaN,1693365,Upper Concourse 425
3,O,NaN,1693477,Upper Concourse 408
4,N,NaN,1693477,Upper Concourse 408
5,P,NaN,1693365,Upper Concourse 425
6,P,NaN,1693365,Upper Concourse 425
7,Q,NaN,1693375,Upper Concourse 410
8,G,"4,5,6,7",1693432,Upper Concourse 421
9,P,NaN,1693355,Upper Concourse 427


In [162]:
listing_df.columns

Index(['businessGuid', 'currentPrice', 'deliveryMethodList',
       'deliveryTypeList', 'dirtyTicketInd', 'faceValue', 'isGA',
       'listingAttributeCategoryList', 'listingAttributeList', 'listingId',
       'listingPrice', 'quantity', 'row', 'score', 'seatNumbers', 'sectionId',
       'sectionName', 'sellerOwnInd', 'sellerSectionName', 'splitOption',
       'splitVector', 'ticketSplit', 'zoneId', 'zoneName'],
      dtype='object')

# SeatGeek

In [3]:
r = requests.get('https://api.seatgeek.com/2/events?listing_count.gt=0&client_id=MTI5MzkxNzN8MTUzNTY3ODY0Ni40Mg')
r.status_code

200

In [4]:
d = json.loads(r.content)
d

{'meta': {'took': 6,
  'per_page': 10,
  'total': 101805,
  'page': 1,
  'geolocation': None},
 'events': [{'type': 'family',
   'description': '',
   'visible_until_utc': '2018-09-05T07:00:00',
   'is_open': False,
   'venue': {'extended_address': 'Las Vegas, NV 89109',
    'name': 'High Roller Wheel At the LINQ',
    'num_upcoming_events': 113,
    'score': 0,
    'timezone': 'America/Los_Angeles',
    'country': 'US',
    'state': 'NV',
    'popularity': 0,
    'has_upcoming_events': True,
    'id': 441839,
    'location': {'lon': -115.168, 'lat': 36.1176},
    'address': '3535 Las Vegas Blvd S',
    'access_method': None,
    'url': 'https://seatgeek.com/venues/high-roller-wheel-at-the-linq/tickets',
    'city': 'Las Vegas',
    'postal_code': '89109',
    'name_v2': 'High Roller Wheel At the LINQ',
    'slug': 'high-roller-wheel-at-the-linq',
    'links': [],
    'display_location': 'Las Vegas, NV'},
   'popularity': 0,
   'time_tbd': True,
   'created_at': '2018-06-20T16:51:34',


In [5]:
e = d.get('events')
ei = iter(e)
len(e)

10

In [6]:
next(ei)

{'type': 'family',
 'description': '',
 'visible_until_utc': '2018-09-05T07:00:00',
 'is_open': False,
 'venue': {'extended_address': 'Las Vegas, NV 89109',
  'name': 'High Roller Wheel At the LINQ',
  'num_upcoming_events': 113,
  'score': 0,
  'timezone': 'America/Los_Angeles',
  'country': 'US',
  'state': 'NV',
  'popularity': 0,
  'has_upcoming_events': True,
  'id': 441839,
  'location': {'lon': -115.168, 'lat': 36.1176},
  'address': '3535 Las Vegas Blvd S',
  'access_method': None,
  'url': 'https://seatgeek.com/venues/high-roller-wheel-at-the-linq/tickets',
  'city': 'Las Vegas',
  'postal_code': '89109',
  'name_v2': 'High Roller Wheel At the LINQ',
  'slug': 'high-roller-wheel-at-the-linq',
  'links': [],
  'display_location': 'Las Vegas, NV'},
 'popularity': 0,
 'time_tbd': True,
 'created_at': '2018-06-20T16:51:34',
 'announce_date': '2018-06-20T00:00:00',
 'id': 4422268,
 'short_title': 'High Roller Wheel - Las Vegas',
 'stats': {'lowest_price_good_deals': None,
  'lowest

In [7]:
curr = next(ei)
curr.get('short_title')
curr.get('stats')

'WNBA Semifinals - Phoenix Mercury at Seattle Storm (Home Game 3, Series Game 5)'

{'lowest_price_good_deals': 47,
 'lowest_price': 47,
 'highest_price': 174,
 'average_price': 103.22,
 'listing_count': 63}